In [1]:
# setup db
from pymongo import MongoClient
import pandas  as pd
import numpy as np


In [284]:
data = pd.read_csv("data/Form.csv")

In [285]:
df = data.iloc[0:,[1,2,3]]

In [286]:
client = MongoClient('localhost',27017)

In [287]:
db = client['Infomations']

In [288]:
collections = db['User']

In [221]:
pipeline = [
        {
          '$lookup':
             {
               'from': "Transactions",
               'localField': "uid",
               'foreignField': "uid",
               'as': "combined_documents"
             }
        },
        {
            '$match': { 'combined_documents.event': "like" }
        }
]

In [225]:
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'gender': { '$first': "$gender" },
            'age': { '$first': "$age" },
            'content_list': {
                '$push': "$combined_documents.Content"
            }
        }
    }
]

In [264]:
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'Gender': { '$first': "$gender" },
            'Age': { '$first': "$age" },
            'Store': {
                '$addToSet': "$combined_documents.Content"
            }
        }
    }
]


In [265]:
cl = list(collections.aggregate(pipeline))

In [228]:
from pprint import pprint

In [216]:
for x in cl:

    pprint([doc['event'] for doc in x['combined_documents']])
    pprint([doc['Content'] for doc in x['combined_documents']])
    pprint(x['name'])
    pprint(x['gender'])
    pprint(x['age'])


    print('')
        # print(x['combined_documents'][0]['event'])
        # print(x['combined_documents'][0]['Content'])


In [217]:
import json

In [266]:
for x in cl:
    pprint(x)

{'Age': 21,
 'Gender': 'Male',
 'Store': ['Nap x Warin', "NAP's Coffee & Roasters"],
 '_id': ObjectId('63d3d51eb0cd665042d3f8e0'),
 'name': 'tong1'}
{'Age': 21,
 'Gender': 'Male',
 'Store': ['Nap x Warin'],
 '_id': ObjectId('63d6163840d228b1ae426045'),
 'name': 'notechu'}


In [267]:
for x in cl:
    print(x)

{'_id': ObjectId('63d3d51eb0cd665042d3f8e0'), 'name': 'tong1', 'Gender': 'Male', 'Age': 21, 'Store': ['Nap x Warin', "NAP's Coffee & Roasters"]}
{'_id': ObjectId('63d6163840d228b1ae426045'), 'name': 'notechu', 'Gender': 'Male', 'Age': 21, 'Store': ['Nap x Warin']}


In [269]:
df1 = pd.DataFrame(cl)

In [270]:
df1 = df1.iloc[0:,[3,2,4]]

In [271]:
df1

,Age,Gender,Store
0,21,Male,"[Nap x Warin, NAP's Coffee & Roasters]"
1,21,Male,[Nap x Warin]


In [273]:
df.append(df1,ignore_index=True)

C:\Users\Note\AppData\Local\Temp\ipykernel_31400\2746747452.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(df1,ignore_index=True)


,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
...,...,...,...
79,28,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
80,42,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
81,25,female,"NAP's Coffee & Roasters, sangob, SongSarn, Nap..."
82,21,Male,"[Nap x Warin, NAP's Coffee & Roasters]"


# raw data

In [280]:
df

,Age,Gender,Store
0,20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
1,16,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA..."
2,22,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ..."
3,21,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop..."
4,19,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
...,...,...,...
77,19,female,"[NAP's Coffee & Roasters, Nap x Warin]"
78,21,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ..."
79,28,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
80,42,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."


# setup data to list

In [316]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [324]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pickle

In [303]:
from pprint import pprint

In [381]:
data = pd.read_csv("data/Form.csv")
df = data.iloc[0:,[1,2,3]]
df

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
...,...,...,...
77,19,female,"NAP's Coffee & Roasters, Nap x Warin"
78,21,female,"NAP's Coffee & Roasters, sangob, SongSarn, LIF..."
79,28,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
80,42,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."


In [334]:
df['Store'] = df['Store'].apply(lambda x: x.split(","))

In [335]:
# ADD USER TO RETRAIN
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'Gender': { '$first': "$gender" },
            'Age': { '$first': "$age" },
            'Store': {
                '$addToSet': "$combined_documents.Content"
            }
        }
    }
]


In [336]:
cl = list(collections.aggregate(pipeline))

In [337]:
df1 = pd.DataFrame(cl)
df1 = df1.iloc[0:,[3,2,4]]
df1["Gender"] = df1["Gender"].apply(lambda x:x.lower()) 

In [338]:
## DATA
df = df.append(df1,ignore_index=True)

C:\Users\Note\AppData\Local\Temp\ipykernel_31400\1881829475.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1,ignore_index=True)


In [339]:
df

,Age,Gender,Store
0,20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
1,16,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA..."
2,22,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ..."
3,21,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop..."
4,19,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
...,...,...,...
79,28,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
80,42,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
81,25,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ..."
82,21,male,"[NAP's Coffee & Roasters, Nap x Warin]"


In [343]:
# setup age
df['age_group'] = df['Age'].apply(lambda x: '10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+')

In [344]:
df

,Age,Gender,Store,age_group
0,20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari...",10-20
1,16,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA...",10-20
2,22,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ...",21-40
3,21,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop...",21-40
4,19,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari...",10-20
...,...,...,...,...
79,28,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and...",21-40
80,42,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and...",41-60
81,25,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ...",21-40
82,21,male,"[NAP's Coffee & Roasters, Nap x Warin]",21-40


In [351]:
age_groups = df.groupby('age_group')
age_gender_rules = {}

# Run the Apriori algorithm and generate association rules for each age group and gender
for age_group, data in age_groups:
    for gender, gender_data in data.groupby('Gender'):
        # Use the TransactionEncoder to convert the data into a suitable format for the Apriori algorithm
        te = TransactionEncoder()
        te_ary = te.fit(gender_data['Store']).transform(gender_data['Store'])
        df_temp = pd.DataFrame(te_ary, columns=te.columns_)

        # Run the Apriori algorithm to find frequent item sets
        frequent_itemsets = apriori(df_temp, min_support=0.05, use_colnames=True)

        # Generate association rules from the frequent item sets
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1)
        # Store the association rules for each age group and gender in the dictionary
        age_gender_rules[(age_group, gender)] = rules

with open('age_gender_rules1.pkl', 'wb') as f:
    pickle.dump(age_gender_rules, f)


In [367]:
with open('age_gender_rules1.pkl', 'rb') as f:
    age_gender_rules = pickle.load(f)




In [461]:
def recommend(user_age_group, user_gender):
    recommended_items = set()
    rules = age_gender_rules[(user_age_group, user_gender)].sort_values(by='confidence', ascending=False)
    for index, row in rules.head(15).iterrows():
        recommended_items.add(row["consequents"])
    recommended_items = list(recommended_items)
    recommended_items = [list(item) for item in recommended_items]
    return recommended_items


In [462]:
gender = input('Gender: ')
z = lambda x: '10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+'
age = z(int(input('Age: ')))
x = recommend(age, gender)



Gender:  male
Age:  20


In [463]:
re = set()
for a in x:
    da = set([ss.strip() for ss in a])
    re.update(da)
    

In [464]:
print(re)

{'Attaroast', 'BalconyKiss Coffee', 'GODFATHER COFFEE', 'sangob', 'Rogue Roasters', 'Saereesook', 'BaanHuakham Cafe & Farmstay', 'Yuanjai Cafe', 'Tree Cafe Rim Moon', 'Bossa cafe', 'Nap x Warin'}


In [465]:
list(re)

['Attaroast',
 'BalconyKiss Coffee',
 'GODFATHER COFFEE',
 'sangob',
 'Rogue Roasters',
 'Saereesook',
 'BaanHuakham Cafe & Farmstay',
 'Yuanjai Cafe',
 'Tree Cafe Rim Moon',
 'Bossa cafe',
 'Nap x Warin']